In [42]:
## Getting content from tavily APIs

query = "How many people die a year from not washing their hands?"

from tavily import TavilyClient
import shutil
import os
import sys

# tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
tavily = TavilyClient(api_key='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')

response = tavily.search(query=query, 
                         search_depth="advanced", 
                         include_raw_content=True,
                         max_results=5)
# response = tavily.get_search_context(query=query, search_depth="advanced", max_tokens=1500)
# response = tavily.qna_search(query=query)
for i, result in enumerate(response['results']):
    print(f"Result {i+1}: {sys.getsizeof(result['raw_content']) / (1024 * 1024)} MB")
    print(f"Length: {len(result['raw_content'])}\n")

response

Result 1: 0.027873992919921875 MB
Length: 14577

Result 2: 0.018556594848632812 MB
Length: 9692

Result 3: 0.022190093994140625 MB
Length: 11597

Result 4: 0.025040626525878906 MB
Length: 26208

Result 5: 0.021535873413085938 MB
Length: 11254



{'query': 'what are llms',
 'follow_up_questions': None,
 'answer': None,
 'images': None,
 'results': [{'title': 'What are LLMs, and how are they used in generative AI?',
   'url': 'https://www.computerworld.com/article/1627101/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html',
   'content': 'LLMs are controlled by parameters, as in millions, billions, and even trillions of them. (Think of a parameter as something that helps an LLM decide between different answer choices.)',
   'score': 0.97196,
   'raw_content': "About\nPolicies\nOur Network\nMore\nAmericas\nAsia\nEurope\nOceania\nTopics\nAbout\nPolicies\nOur Network\nMore\nWhat are LLMs, and how are they used in generative AI?\nLarge language models are the algorithmic basis for chatbots like OpenAI's ChatGPT and Google's Bard. The technology is tied back to billions — even trillions — of parameters that can make them both inaccurate and non-specific for vertical industry use. Here's what LLMs are and how t

In [2]:
# if os.path.exists('tavily_data'):
#     shutil.rmtree('tavily_data')
#     os.makedirs('tavily_data')
# else:
#     os.makedirs('tavily_data')

meta_map = {}

for count, result in enumerate(response['results']):
    f = open("tavily_data/file{}.txt".format(count), "a", encoding="utf-8")
    meta_map[count] = {'title':result['title'], 'url':result['url']}
    f.truncate(0)
    f.write(result['raw_content'])
    f.close()

In [3]:
## querying AI model

from pathlib import Path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from typing import List, Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pydantic import BaseModel
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.chat_engine import CondenseQuestionChatEngine

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.core import Settings
from llama_index.core import set_global_tokenizer


HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

Settings.chunk_size = 256
gpt4 = OpenAI(model="gpt-4", api_key=os.environ['OPENAI_API_KEY'])
gpt = OpenAI(model="gpt-3.5-turbo")
Settings.llm = gpt

#set the embed model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
#Settings.embed_model = OpenAIEmbedding()
#https://huggingface.co/BAAI/bge-small-en-v1.5?library=sentence-transformers

In [6]:
documents = SimpleDirectoryReader("tavily_data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [7]:
base_retriever = index.as_retriever(similarity_top_k=5)
retrievals = base_retriever.retrieve(query)
for n in retrievals:
    #if (not n.text.__contains__("http")):
        display_source_node(n, source_length=1500)




**Node ID:** 6f8ad8f4-aa56-489c-b66c-81710a617d9b<br>**Similarity:** 0.895583089618943<br>**Text:** By examining specific parenting practices that are amenable to change, such as parent involvement, and the mechanisms by which these practices influence academic performance, programs may be developed to increase a child's academic performance. While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003). Understanding these mechanisms would inform further research and policy initiatives and may lead to the development of more effective intervention programs designed to increase children's academic performance.
Models of Parent Involvement
Parent involvement has been defined and measured in multiple ways, including activities that parents engage in at home and at school and positive attitudes parents have towards their child's education, school, and teacher (Epstein, 1996; Grolnick & Slowiaczek, 1994; Kohl, Lengua, & McMahon, 2000). The distinction between the activities parents partake in and the attitude parents have towards education was highlighted by several recent studies.<br>

**Node ID:** 94ca789d-3f3a-4b51-8fee-eb318c707a6e<br>**Similarity:** 0.8857707416042999<br>**Text:** Therefore, it is important to examine factors that contribute to early academic success and that are amenable to change.
Researchers have reported that parent-child interactions, specifically stimulating and responsive parenting practices, are important influences on a child's academic development (Christian, Morrison, & Bryant, 1998; Committee on Early Childhood Pedagogy, 2000). By examining specific parenting practices that are amenable to change, such as parent involvement, and the mechanisms by which these practices influence academic performance, programs may be developed to increase a child's academic performance. While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003). Understanding these mechanisms would inform further research and policy initiatives and may lead to the development of more effective intervention programs designed to increase children's academic performance.
Models of Parent Involvement
Parent involvement has been defined and measured in multiple ways, including activities that parents engage in at home and at school and positive attitudes parents have towards their child's education, school, and teacher (Epstein, 1996; Grolnick & Slowiaczek, 1994; Kohl, Lengua, & McMahon, 2000).<br>

**Node ID:** 8d2dc076-f878-48eb-b177-2df9b50f93be<br>**Similarity:** 0.8823156343871914<br>**Text:** Further, parent involvement was significantly related to academic performance above and beyond the impact of the child's intelligence (IQ), a variable not accounted for in previous research.
Findings from the present study demonstrated that increased parent involvement is significantly related to a child's increased perception of cognitive competence. This finding is consistent with previous studies (Gonzalez-DeHass, Willems, & Holbein, 2005; Grolnick, Ryan, & Deci, 1991). While outside the scope of the present study, it is conceivable that parent involvement may influence the child's perception of cognitive competence by means described by Bandura (1977). Findings demonstrated that increased parent involvement was significantly related to increased quality of the student-teacher relationship. Findings also demonstrated that increased perceived cognitive competence was related to higher achievement test scores and that the quality of the student-teacher relationship was significantly related to the child's academic performance, measured by both standardized achievement test scores and the child's classroom academic performance.<br>

**Node ID:** 27ac7a53-aa5f-417f-991a-80aa3cac81ee<br>**Similarity:** 0.8802547576874705<br>**Text:** While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003). Understanding these mechanisms would inform further research and policy initiatives and may lead to the development of more effective intervention programs designed to increase children's academic performance.
Models of Parent Involvement
Parent involvement has been defined and measured in multiple ways, including activities that parents engage in at home and at school and positive attitudes parents have towards their child's education, school, and teacher (Epstein, 1996; Grolnick & Slowiaczek, 1994; Kohl, Lengua, & McMahon, 2000). The distinction between the activities parents partake in and the attitude parents have towards education was highlighted by several recent studies. Several studies found that increased frequency of activities was associated with higher levels of child misbehavior in the classroom (Izzo, Weissberg, Kasprow, & Fendrich, 1999), whereas positive attitudes towards education and school were associated with the child's increased academic performance (Rimm-Kaufman, Pianta, Cox, & Bradley, 2003).<br>

**Node ID:** 109c3238-79cb-420a-be66-9f4e6b2f961f<br>**Similarity:** 0.8797144884821717<br>**Text:** Here are five takeaways from the research.
1. Studies show more parental involvement leads to improved academic outcomes
When parents are involved in their children’s schooling, students show higher academic achievement, school engagement, and motivation, according to a 2019 American Psychological Association review of 448 independent studies on parent involvement.
A 2005 study from researchers at the Johns Hopkins University Center on School, Family and Community Partnerships, for example, showed that school practices encouraging families to support their child’s math learning at home led to higher percentages of students scoring at or above proficiency on standardized math tests.
And research shows that parent involvement with reading activities has a positive impact on reading achievement, language comprehension, and expressive language skills, as well as students’ interest in reading, attitudes toward reading, and level of attention in the classroom, according to a research summary by the National Literacy Trust.<br>

In [145]:
query_engine = RetrieverQueryEngine.from_args(base_retriever, streaming=True) 
chat_engine = CondenseQuestionChatEngine.from_defaults(query_engine=query_engine)
response = chat_engine.chat(query)
print(response)
# for text in response.response_gen:
#     response.print_response_stream()

Parental involvement has been consistently linked to a child's increased academic performance. Studies have shown that when parents are actively engaged in their children's education by participating in school activities, volunteering, supporting learning at home, and maintaining positive attitudes towards education, students tend to exhibit higher academic achievement, school engagement, motivation, improved reading skills, language comprehension, and expressive language skills. Additionally, parental involvement has been associated with positive social and emotional outcomes for students, such as higher self-esteem, enjoyment of school, and decreased instances of delinquency. This involvement not only benefits the students directly but also equips teachers with a better understanding of their students' personal situations and challenges, enabling them to provide more effective support in the classroom.


In [24]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
text_splitter = SentenceSplitter()
embed_model = OpenAIEmbedding()

Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)

sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)


In [8]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(query)

for window in window_response.source_nodes:
    print("Window: ")
    print(window.node.metadata["window"])
    print("Original Sentence: ")
    print(window.node.metadata["original_text"])
    print()

print(window_response)


NameError: name 'sentence_index' is not defined